<a href="https://colab.research.google.com/github/Rauloigs/mate_est_finanzas/blob/main/tarea_baseball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librería

In [24]:
# Import Libraries
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import t
import datetime as datetime
import gspread

# To upload files from drive
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from google.auth import default
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')


# Authenticate and create the PyDrive client.
creds, _ = default()
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Desacarga del Archivo

todos los datos tienen formato correcto: float e int

In [25]:
link = 'https://drive.google.com/file/d/1ASD5JPahYnS601MnKIZo1aoUXEr89kw0/view?usp=sharing'
# to get the id part of the file
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('baseball_22.csv')
baseball = pd.read_csv('baseball_22.csv')
print(f"baseball: {baseball.shape}")
print(f" There are: {baseball.isna().sum().sum()} NaNs")
baseball.head()

baseball: (30, 11)
 There are: 0 NaNs


,Team,League,Year Stadium Opened,Team Salary ($ mil),Attendance,Wins,ERA,BA,HR,Net worth ($ billions),2022 Playoffs
0,Arizona Diamondbacks,National,1998,66.468,1605199,74,4.25,0.230,173,1.380,0
1,Atlanta Braves,National,2017,150.310,3129931,101,3.46,0.253,243,2.100,1
2,Baltimore Orioles,American,1992,24.555,1368367,83,3.97,0.236,171,1.375,0
3,Boston Red Sox,American,1912,152.141,2625089,78,4.53,0.258,155,3.900,0
4,Chicago Cubs,National,1914,114.050,2616780,74,4.00,0.238,159,3.800,0


#Funciones

Al tener una σ desconocida, lo ideal es utilizar la distribución t para calcular el intervalo de confianza, sin embargo, también es posible utilizar una distribución normal dado que n=≥30

**Funciones**
1. intervalo_confianza_z
2. intervalo_confianza_t

**Resultado de las funciones son dos cosas**
- El intervalo
- El valor de Z

In [26]:
# función para calcular el intervalo de confianza Z
def intervalo_confianza_z(media, desviacion_estandar_poblacional, n, confianza):
  """
  Calcula el intervalo de confianza para una distribución normal Z.
  """
  error_estandar = desviacion_estandar_poblacional / np.sqrt(n)
  valor_critico = st.norm.ppf((1 + confianza) / 2) #valor de z
  margen_error = valor_critico * error_estandar
  intervalo_confianza = (media - margen_error, media + margen_error)
  return print("El intervalo es:", intervalo_confianza, "Con un valor Z de:",valor_critico)

def intervalo_confianza_t(media, desviacion_estandar_poblacional, n, confianza):
  """
  Calcula el intervalo de confianza para una distribución normal Z.
  """
  error_estandar = desviacion_estandar_poblacional / np.sqrt(n)
  valor_critico = t.ppf((1+0.95)/2, n-1) #valor de z
  margen_error = valor_critico * error_estandar
  intervalo_confianza = (media - margen_error, media + margen_error)
  return print("El intervalo es:", intervalo_confianza, "Con un valor t de:",valor_critico)


## 1. Obtenga un intervalo de confianza de 95% para la media de homeruns por equipo

se usa la distribución normal z, porque n >= 30

In [28]:
# ¿Promedio de los HR?
media_hr = baseball['HR'].mean()
media_hr

173.83333333333334

In [29]:
#¿La desv estándar de los HR?
desv_std_hr = baseball['HR'].std()
desv_std_hr

34.87325161439523

In [30]:
# tamaño de la muestra
# este tamaño de muestra se utiliza en los otros dos problemas!
# con la función t student n = n-1 automáticamente
n = baseball['HR'].count()
n

30

##Respuesta

In [38]:
# Calcula el intervalo de confianza
intervalo_t = intervalo_confianza_t(media_hr, desv_std_hr, n, confianza=0.95)
intervalo_t


El intervalo es: (160.81144717180885, 186.85521949485783) Con un valor t de: 2.045229642132703


In [42]:
# Calcula el intervalo de confianza
intervalo_z = intervalo_confianza_z(media_hr, desv_std_hr, n, confianza=0.95)
intervalo_z

El intervalo es: (161.35432982116316, 186.31233684550352) Con un valor Z de: 1.959963984540054


##2. Obtenga un intervalo de confianza del 95% para la media del promedio de bateo de los equipos de la liga

In [32]:
# ¿Promedio de los HR?
media_ba = baseball['BA'].mean()
media_ba

0.24263333333333334

In [33]:
#¿La desv estándar de los HR?
desv_std_ba = baseball['BA'].std()
desv_std_ba

0.011943785957078507

##Respuesta

In [39]:
# Calcula el intervalo de confianza
intervalo_t = intervalo_confianza_t(media_ba, desv_std_ba, n, confianza=0.95)
intervalo_t

El intervalo es: (0.238173450360835, 0.24709321630583167) Con un valor t de: 2.045229642132703


In [43]:
# Calcula el intervalo de confianza
intervalo_z = intervalo_confianza_z(media_ba, desv_std_ba, n, confianza=0.95)
intervalo_z

El intervalo es: (0.23835938295569012, 0.24690728371097656) Con un valor Z de: 1.959963984540054


##3. Obtenga un intervalo de confianza del 95% para la media de carreras obtenidas por los equipo de la liga

In [35]:
# ¿Promedio de los HR?
media_wins = baseball['ERA'].mean()
media_wins

3.9653333333333336

In [36]:
#¿La desv estándar de los HR?
desv_std_wins = baseball['ERA'].std()
desv_std_wins

0.5591787903168627

##Respuesta

In [40]:
# Calcula el intervalo de confianza
intervalo_t = intervalo_confianza_t(media_wins, desv_std_wins, n, confianza=0.95)
intervalo_t

El intervalo es: (3.7565325414840465, 4.174134125182621) Con un valor t de: 2.045229642132703


In [44]:
# Calcula el intervalo de confianza
intervalo_z = intervalo_confianza_z(media_wins, desv_std_wins, n, confianza=0.95)
intervalo_z

El intervalo es: (3.7652374499481787, 4.165429216718489) Con un valor Z de: 1.959963984540054
